<a href="https://colab.research.google.com/github/EstebanUOC/IA-Models/blob/main/exercise_generation_mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pathlib
import random
from collections import defaultdict
import json

In [2]:
np.random.seed(1)

In [3]:
verbose = True

In [4]:
n_combinations_per_word = 2

In [5]:
basepath = "../../data/"

In [6]:
basepath = pathlib.Path(basepath)

In [7]:
syllable_type = "Directa"

In [8]:
if syllable_type == "Directa":
    syllable_type_char = "D"
elif syllable_type == "Inversa":
    syllable_type_char = "I"
elif syllable_type == "Travada":
    syllable_type_char = "B"
else:
    print(f"Invalid syllable_type. Accepted 'Directa', 'Inversa', 'Travada'. Received '{syllable_type}'")
    raise ValueError

In [9]:
sounds = [
    "Nasal sonora M",
    "Fricativa sorda F",
    "Nasal sonora N",
    "Lateral sonora L",
    "Vibrant sonora R (r, rr)",
    "Bategant sonora R",
    "Lateral sonora LL",
    "Nasal sonora NY",
    "Fricatiu sord (s, ss, c, ç)",
    "Fricatives sonores (z, s)",
    "Fricativa sonora ʒ (J, G)",
    "Fricativa sorda ʃ (x)",
    "Africada sonora dʒ (tg, tj)",
    "Africada sonora dz (tz, zz)",
    "Africada sorda tʃ (tx, ig)",
    "Oclusiva sonora B (b, v)",
    "Oclusiva sorda P",
    "Oclusiva sonora D",
    "Oclusiva sorda T",
    "Oclusiva sonora G",
    "Oclusiva sorda K (c,qu,k)",
]

In [10]:
def remove_accents(word):
    word = word.replace("à", "a")
    word = word.replace("è", "e")
    word = word.replace("é", "e")
    word = word.replace("í", "i")
    word = word.replace("ï", "i")
    word = word.replace("ò", "o")
    word = word.replace("ó", "o")
    word = word.replace("ú", "u")
    word = word.replace("ü", "u")

    return word

In [ ]:
def extract_words_by_sound_syllable_position_and_count(sound: str, syllable_position: str, syllable_count: int):
    if syllable_position == "Inici":
        key_syllable_position = 0
    elif syllable_position == "Meitat":
        key_syllable_position = 1
    elif syllable_position == "Final":
        key_syllable_position = -1
    else:
        print(f"Invalid syllable_position. Accepted 'Inici', 'Meitat', 'Final'. Received '{syllable_position}'")
        return None

    if syllable_type == "Directa":
        syllable_type_suffix = "Dir"
    elif syllable_type == "Inversa":
        syllable_type_suffix = "Inv"
    elif syllable_type == "Travada":
        syllable_type_suffix = "Tra"
    else:
        print(f"Invalid syllable_type. Accepted 'Directa', 'Inversa', 'Travada'. Received '{syllable_type}'")
        return None

    if verbose:
        print(f"Reading words for sound {sound}, syllable_position {syllable_position} and syllable count {syllable_count}")

    words_df = pd.read_excel(basepath / "paraules_picofon.xlsx", sheet_name=sound, header=1)
    words_df.columns = [c.strip() for c in words_df.columns]

    words_syllables = defaultdict(set)
    try:
        words_df = words_df.loc[:, f"{syllable_position} {syllable_count}S{syllable_type_suffix}"]
        words = words_df.dropna().to_list()

        for w in words:
            w = w.split("/")[0].strip()
            w = w.split("(")[0].strip()
            w_syllables = w.split("#")
            if len(w_syllables) != syllable_count:
                continue

            key_syllable = w_syllables[key_syllable_position]
            key_syllable = remove_accents(key_syllable)
            words_syllables[key_syllable].add(w)

        for syllable, words in words_syllables.items():
            words_syllables[syllable] = list(words)

        n_words = sum([len(words) for words in words_syllables.values()])

        if verbose:
            print(f"Found {n_words} valid words four sound {sound} in position {syllable_position}, with type {syllable_type} and with {syllable_count} syllables")
    except KeyError:
        if verbose:
            print(f"No words four sound {sound} in position {syllable_position}, with type {syllable_type} and with {syllable_count} syllables")

    return words_syllables

In [ ]:
def extract_words():
    all_words = {}
    for sound in sounds:
        all_words[sound] = {}
        for syllable_position in ["Inici", "Meitat", "Final"]:
            all_words[sound][syllable_position] = {}
            for syllable_count in range(1, 6):
                all_words[sound][syllable_position][syllable_count] = extract_words_by_sound_syllable_position_and_count(sound, syllable_position, syllable_count)

    return all_words

In [ ]:
all_words = extract_words()

In [ ]:
all_words_list = []
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        for sound in sounds:
            for syllable, words in all_words[sound][syllable_position][syllable_count].items():
                all_words_list += [remove_accents(w.replace("#", "")) for w in words]

all_words_set = set(all_words_list)

In [ ]:
if verbose:
    print(f"Number of words: {len(all_words)}")
    print(all_words)

In [ ]:
exercise_id = 0
def generate_exercise_id():
    global exercise_id
    exercise_id += 10
    return exercise_id

In [ ]:
def compute_initial_exercise_id(base_initial_exercise_id):
    if syllable_type == "Directa":
        return base_initial_exercise_id

    if syllable_type == "Inversa":
        return base_initial_exercise_id + 100_000_000

    if syllable_type == "Travada":
        return base_initial_exercise_id + 200_000_000

In [ ]:
def find_repeated_exercise(exercise, exercises, keys_to_compare):
    for ex in exercises:
        equal_exercise = True
        for key in keys_to_compare:
            if ex[key] != exercise[key]:
                equal_exercise = False
                break

        if equal_exercise:
            return True

    return False

In [ ]:
def select_random_word(sounds_to_avoid, syllable_position, syllable_count, main_word, force_similarity=False):
    n_tries = 0
    while n_tries < 30:
        n_tries += 1
        try:
            selected_sound = np.random.choice([s for s in sounds if s not in sounds_to_avoid], size=1)[0]
            selected_syllable = np.random.choice(list(all_words[selected_sound][syllable_position][syllable_count].keys()), size=1)[0]
            selected_word = np.random.choice(all_words[selected_sound][syllable_position][syllable_count][selected_syllable], size=1)[0]

            if force_similarity:
                main_word_syllables = main_word.split("#")
                selected_word_syllables = selected_word.split("#")
                equal_syllables = 0
                for i in range(syllable_count):
                    if main_word_syllables[i] == selected_word_syllables[i]:
                        equal_syllables += 1

                if n_tries < 10:
                    if equal_syllables < syllable_count - 1:
                        continue

                if n_tries < 20:
                    if equal_syllables < syllable_count - 2:
                        continue

            return selected_word
        except ValueError:
            pass

    return None

## Identificar síl·labes

### Jutjar

In [ ]:
def identificar_jutjar(syllable_position, syllable_count):
    if syllable_position == "Inici":
        skill = "ISI"
    elif syllable_position == "Meitat":
        skill = "ISM"
    elif syllable_position == "Final":
        skill = "ISF"

    exercises = []
    for sound in sounds:
        for syllable, words in all_words[sound][syllable_position][syllable_count].items():
            if len(words) == 1:
                continue

            for word in words:
                n_combinations = 0
                n_tries = 0
                while n_combinations < n_combinations_per_word and n_tries < 2*n_combinations_per_word:
                    n_tries += 1

                    exercise_si = {}
                    exercise_si["response"] = "yes"
                    exercise_si["language"] = "CA"
                    exercise_si["unit"] = "SY"
                    exercise_si["skill"] = skill
                    exercise_si["game_mechanic"] = "J"
                    exercise_si["syllables_number"] = syllable_count
                    exercise_si["syllable_structure"] = syllable_type_char
                    exercise_si["sound"] = sound
                    selected_word = np.random.choice([w for w in words if w != word], size=1)[0]
                    exercise_si["main_word"] = word.replace("#", "")
                    exercise_si["option1"] = selected_word.replace("#", "")

                    exercise_no = {}
                    exercise_no["response"] = "no"
                    exercise_no["language"] = "CA"
                    exercise_no["unit"] = "SY"
                    exercise_no["skill"] = skill
                    exercise_no["game_mechanic"] = "J"
                    exercise_no["syllables_number"] = syllable_count
                    exercise_no["syllable_structure"] = syllable_type_char
                    exercise_no["sound"] = sound
                    selected_word = select_random_word([sound], syllable_position, syllable_count, word, force_similarity=True)
                    if selected_word is None:
                        continue
                    exercise_no["main_word"] = word.replace("#", "")
                    exercise_no["option1"] = selected_word.replace("#", "")

                    if not find_repeated_exercise(exercise_si, exercises, keys_to_compare=["main_word", "option1"]) and not find_repeated_exercise(exercise_no, exercises, keys_to_compare=["main_word", "option1"]):
                        exercise_si["id"] = generate_exercise_id()
                        exercises.append(exercise_si)

                        exercise_no["id"] = generate_exercise_id()
                        exercises.append(exercise_no)

                        n_combinations += 1

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(101_000_000)
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        exercicis_identificar_jutjar = identificar_jutjar(syllable_position, syllable_count)
        if exercicis_identificar_jutjar:
            print(f"Number of exercises for position {syllable_position} and syllable count {syllable_count}: {len(exercicis_identificar_jutjar)}")

            with open(basepath / f"identificar_jutjar_{syllable_position}_{syllable_type}_{syllable_count}s.json", "w", encoding="utf-8") as f:
                json.dump(exercicis_identificar_jutjar, f, ensure_ascii=False, indent=2, sort_keys=True)

### Seleccionar la que no encaixa

In [ ]:
def identificar_seleccionar(syllable_position, syllable_count):
    if syllable_position == "Inici":
        skill = "ISI"
    elif syllable_position == "Meitat":
        skill = "ISM"
    elif syllable_position == "Final":
        skill = "ISF"

    exercises = []
    for sound in sounds:
        for syllable, words in all_words[sound][syllable_position][syllable_count].items():
            if len(words) < 4:
                continue

            for word in words:
                n_combinations = 0
                n_tries = 0
                while n_combinations < n_combinations_per_word and n_tries < 2*n_combinations_per_word:
                    n_tries += 1

                    exercise = {}
                    selected_words = list(np.random.choice([w.replace("#", "") for w in words if w != word], size=2, replace=False))
                    wrong_word = select_random_word([sound], syllable_position, syllable_count, word)
                    if wrong_word is None:
                        continue

                    exercise_words = [word.replace("#", ""), wrong_word.replace("#", "")] + selected_words
                    random.shuffle(exercise_words)

                    exercise["main_word"] = word.replace("#", "")
                    exercise["option1"] = exercise_words[0]
                    exercise["option2"] = exercise_words[1]
                    exercise["option3"] = exercise_words[2]
                    exercise["option4"] = exercise_words[3]
                    exercise["response"] = wrong_word.replace("#", "")
                    exercise["language"] = "CA"
                    exercise["unit"] = "SY"
                    exercise["skill"] = skill
                    exercise["game_mechanic"] = "S"
                    exercise["syllables_number"] = syllable_count
                    exercise["syllable_structure"] = syllable_type_char
                    exercise["sound"] = sound

                    if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word", "option1", "option2", "option3", "option4"]):
                        exercise["id"] = generate_exercise_id()
                        exercises.append(exercise)

                        n_combinations += 1

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(102_000_000)
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        exercicis_identificar_seleccionar = identificar_seleccionar(syllable_position, syllable_count)
        if exercicis_identificar_seleccionar:
            print(f"Number of exercises for position {syllable_position} and syllable count {syllable_count}: {len(exercicis_identificar_seleccionar)}")

            with open(basepath / f"identificar_seleccionar_{syllable_position}_{syllable_type}_{syllable_count}s.json", "w", encoding="utf-8") as f:
                json.dump(exercicis_identificar_seleccionar, f, ensure_ascii=False, indent=2, sort_keys=True)

### Relacionar

In [ ]:
def identificar_relacionar(syllable_position, syllable_count):
    if syllable_position == "Inici":
        skill = "ISI"
    elif syllable_position == "Meitat":
        skill = "ISM"
    elif syllable_position == "Final":
        skill = "ISF"

    exercises = []
    for sound in sounds:
        for syllable, words in all_words[sound][syllable_position][syllable_count].items():
            if len(words) == 1:
                continue

            for word in words:
                n_combinations = 0
                n_tries = 0
                while n_combinations < n_combinations_per_word and n_tries < 2*n_combinations_per_word:
                    n_tries += 1

                    exercise = {}
                    selected_word = np.random.choice([w for w in words if w != word], size=1)[0]

                    wrong_word_1 = select_random_word([sound], syllable_position, syllable_count, word)
                    wrong_word_2 = select_random_word([sound], syllable_position, syllable_count, word)
                    wrong_word_3 = select_random_word([sound], syllable_position, syllable_count, word)

                    if wrong_word_1 is None or wrong_word_2 is None or wrong_word_3 is None:
                        continue

                    exercise_words = [word.replace("#", ""), selected_word.replace("#", ""), wrong_word_1.replace("#", ""), wrong_word_2.replace("#", ""), wrong_word_3.replace("#", "")]
                    random.shuffle(exercise_words)

                    exercise["main_word"] = word.replace("#", "")
                    exercise["option1"] = exercise_words[0]
                    exercise["option2"] = exercise_words[1]
                    exercise["option3"] = exercise_words[2]
                    exercise["option4"] = exercise_words[3]
                    exercise["option5"] = exercise_words[4]
                    exercise["response"] = selected_word.replace("#", "")
                    exercise["language"] = "CA"
                    exercise["unit"] = "SY"
                    exercise["skill"] = skill
                    exercise["game_mechanic"] = "R"
                    exercise["syllables_number"] = syllable_count
                    exercise["syllable_structure"] = syllable_type_char
                    exercise["sound"] = sound

                    if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word", "option1", "option2", "option3", "option4", "option5"]):
                        exercise["id"] = generate_exercise_id()
                        exercises.append(exercise)

                        n_combinations += 1

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(103_000_000)
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        exercicis_identificar_relacionar = identificar_relacionar(syllable_position, syllable_count)
        if exercicis_identificar_relacionar:
            print(f"Number of exercises for position {syllable_position} and syllable count {syllable_count}: {len(exercicis_identificar_relacionar)}")

            with open(basepath / f"identificar_relacionar_{syllable_position}_{syllable_type}_{syllable_count}s.json", "w", encoding="utf-8") as f:
                json.dump(exercicis_identificar_relacionar, f, ensure_ascii=False, indent=2, sort_keys=True)

## Comptar síl·labes

### Jutjar

In [ ]:
def comptar_jutjar(syllable_position, syllable_count):
    exercises = []
    for sound in sounds:
        sound_words = []
        for syllable, words in all_words[sound][syllable_position][syllable_count].items():
            sound_words += words

        if sound_words:
            exercise = {}
            exercise["words"] = [word.replace("#", "") for word in sound_words]
            exercise["language"] = "CA"
            exercise["unit"] = "SY"
            exercise["skill"] = "SPS"
            exercise["game_mechanic"] = "J"
            exercise["syllables_number"] = syllable_count
            exercise["syllable_structure"] = syllable_type_char
            exercise["sound"] = sound
            exercise["id"] = generate_exercise_id()
            exercises.append(exercise)

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(104_000_000)
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        exercicis_comptar_jutjar = comptar_jutjar(syllable_position, syllable_count)
        if exercicis_comptar_jutjar:
            print(f"Number of exercises for position {syllable_position} and syllable count {syllable_count}: {len(exercicis_comptar_jutjar)}")

            with open(basepath / f"comptar_jutjar_{syllable_position}_{syllable_type}_{syllable_count}s.json", "w", encoding="utf-8") as f:
                json.dump(exercicis_comptar_jutjar, f, ensure_ascii=False, indent=2, sort_keys=True)

## Combinar síl·labes

### Jutjar

In [ ]:
def combinar_jutjar():
    exercises = []
    monosyllable_words = []
    bisyllable_words = []
    for sound in sounds:
        for syllable, words in all_words[sound]["Inici"][1].items():
            monosyllable_words += [remove_accents(w.replace("#", "")) for w in words]

        for syllable, words in all_words[sound]["Inici"][2].items():
            bisyllable_words += [remove_accents(w.replace("#", "")) for w in words]

    for w1 in monosyllable_words:
        for w2 in monosyllable_words:

            if f"{w1}{w2}" in bisyllable_words:
                exercise = {}
                exercise["main_word"] = f"{w1}{w2}"
                exercise["target1"] = w1
                exercise["target2"] = w2
                exercise["response"] = f"{w1}{w2}"
                exercise["language"] = "CA"
                exercise["unit"] = "SY"
                exercise["skill"] = "CS"
                exercise["game_mechanic"] = "J"
                exercise["syllables_number"] = syllable_count
                exercise["syllable_structure"] = syllable_type_char
                exercise["sound"] = sound

                if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word"]):
                    exercise["id"] = generate_exercise_id()
                    exercises.append(exercise)

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(105_000_000)
exercicis_combinar_jutjar = combinar_jutjar()
if exercicis_combinar_jutjar:
    print(f"Number of exercises: {len(exercicis_combinar_jutjar)}")

    with open(basepath / f"combinar_jutjar_{syllable_type}.json", "w", encoding="utf-8") as f:
        json.dump(exercicis_combinar_jutjar, f, ensure_ascii=False, indent=2, sort_keys=True)

### Crear juntar

In [ ]:
def combinar_crearjuntar(syllable_position, syllable_count):
    exercises = []
    for sound in sounds:
        for syllable, words in all_words[sound][syllable_position][syllable_count].items():
            for word in words:
                exercise = {}
                exercise["main_word"] = word.replace("#", "")
                exercise["words"] = word.split("#")
                exercise["response"] = word.replace("#", "")
                exercise["language"] = "CA"
                exercise["unit"] = "SY"
                exercise["skill"] = "CS"
                exercise["game_mechanic"] = "CJ"
                exercise["syllables_number"] = syllable_count
                exercise["syllable_structure"] = syllable_type_char
                exercise["sound"] = sound
                if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word"]):
                    exercise["id"] = generate_exercise_id()
                    exercises.append(exercise)

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(106_000_000)
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        exercicis_combinar_relacionar = combinar_crearjuntar(syllable_position, syllable_count)
        if exercicis_combinar_relacionar:
            print(f"Number of exercises for position {syllable_position} and syllable count {syllable_count}: {len(exercicis_combinar_relacionar)}")

            with open(basepath / f"combinar_crearjuntar_{syllable_position}_{syllable_type}_{syllable_count}s.json", "w", encoding="utf-8") as f:
                json.dump(exercicis_combinar_relacionar, f, ensure_ascii=False, indent=2, sort_keys=True)

## Omissió de síl·labes

### Jutjar

In [ ]:
def omissio_jutjar(syllable_position, syllable_count):
    exercises = []
    for sound in sounds:
        for syllable, words in all_words[sound][syllable_position][syllable_count].items():
            for word in words:
                for i in range(syllable_count):
                    word_syllables = word.split("#")
                    word_syllables.pop(i)
                    new_word = "".join(word_syllables)

                    if new_word in all_words_set:
                        exercise = {}
                        exercise["main_word"] = word.replace("#", "")
                        exercise["response"] = new_word
                        exercise["language"] = "CA"
                        exercise["unit"] = "SY"
                        exercise["skill"] = "OS"
                        exercise["game_mechanic"] = "J"
                        exercise["syllables_number"] = syllable_count
                        exercise["syllable_structure"] = syllable_type_char
                        exercise["sound"] = sound
                        if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word", "response"]):
                            exercise["id"] = generate_exercise_id()
                            exercises.append(exercise)

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(107_000_000)
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        exercicis_omissio_jutjar = omissio_jutjar(syllable_position, syllable_count)
        if exercicis_omissio_jutjar:
            print(f"Number of exercises for position {syllable_position} and syllable count {syllable_count}: {len(exercicis_omissio_jutjar)}")

            with open(basepath / f"omissio_jutjar_{syllable_position}_{syllable_type}_{syllable_count}s.json", "w", encoding="utf-8") as f:
                json.dump(exercicis_omissio_jutjar, f, ensure_ascii=False, indent=2, sort_keys=True)

### Crear

In [ ]:
def omissio_crear(syllable_position, syllable_count):
    exercises = []
    for sound in sounds:
        for syllable, words in all_words[sound][syllable_position][syllable_count].items():
            for word in words:
                for i in range(syllable_count):
                    word_syllables = word.split("#")
                    word_syllables.pop(i)
                    new_word = "".join(word_syllables)

                    if new_word:
                        exercise = {}
                        exercise["main_word"] = word.replace("#", "")
                        exercise["response"] = new_word
                        exercise["language"] = "CA"
                        exercise["unit"] = "SY"
                        exercise["skill"] = "OS"
                        exercise["game_mechanic"] = "C"
                        exercise["syllables_number"] = syllable_count
                        exercise["syllable_structure"] = syllable_type_char
                        exercise["sound"] = sound
                        if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word", "response"]):
                            exercise["id"] = generate_exercise_id()
                            exercises.append(exercise)

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(108_000_000)
for syllable_position in ["Inici", "Meitat", "Final"]:
    for syllable_count in range(1, 6):
        exercicis_omissio_crear = omissio_crear(syllable_position, syllable_count)
        if exercicis_omissio_crear:
            print(f"Number of exercises for position {syllable_position} and syllable count {syllable_count}: {len(exercicis_omissio_crear)}")

            with open(basepath / f"omissio_crear_{syllable_position}_{syllable_type}_{syllable_count}s.json", "w", encoding="utf-8") as f:
                json.dump(exercicis_omissio_crear, f, ensure_ascii=False, indent=2, sort_keys=True)

## Inversió de síl·labes

### Jutjar

In [ ]:
def inversio_jutjar():
    exercises = []
    bisyllable_words = []
    for sound in sounds:
        for syllable, words in all_words[sound]["Inici"][2].items():
            bisyllable_words += [remove_accents(w) for w in words]

    bisyllable_words_normalized = [w.replace("#", "") for w in bisyllable_words]

    for w in bisyllable_words:
        w1, w2 = w.split("#")

        if f"{w2}{w1}" in bisyllable_words_normalized:
            exercise = {}
            exercise["main_word"] = w.replace("#", "")
            exercise["response"] = f"{w2}{w1}"
            exercise["language"] = "CA"
            exercise["unit"] = "SY"
            exercise["skill"] = "ISP"
            exercise["game_mechanic"] = "J"
            exercise["syllables_number"] = syllable_count
            exercise["syllable_structure"] = syllable_type_char
            exercise["sound"] = sound

            if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word", "response"]):
                exercise["id"] = generate_exercise_id()
                exercises.append(exercise)

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(109_000_000)
exercicis_inversio_jutjar = inversio_jutjar()
if exercicis_inversio_jutjar:
    print(f"Number of exercises: {len(exercicis_inversio_jutjar)}")

    with open(basepath / f"inversio_jutjar_{syllable_type}.json", "w", encoding="utf-8") as f:
        json.dump(exercicis_inversio_jutjar, f, ensure_ascii=False, indent=2, sort_keys=True)

### Crear

In [ ]:
def inversio_crear():
    exercises = []
    bisyllable_words = []
    for sound in sounds:
        for syllable, words in all_words[sound]["Inici"][2].items():
            for word in words:
                w1, w2 = word.split("#")

                exercise = {}
                exercise["main_word"] = word.replace("#", "")
                exercise["response"] = f"{w2}{w1}"
                exercise["language"] = "CA"
                exercise["unit"] = "SY"
                exercise["skill"] = "ISP"
                exercise["game_mechanic"] = "C"
                exercise["syllables_number"] = syllable_count
                exercise["syllable_structure"] = syllable_type_char
                exercise["sound"] = sound


                if not find_repeated_exercise(exercise, exercises, keys_to_compare=["main_word", "response"]):
                    exercise["id"] = generate_exercise_id()
                    exercises.append(exercise)

    return exercises

In [ ]:
exercise_id = compute_initial_exercise_id(110_000_000)
exercicis_inversio_crear = inversio_crear()
if exercicis_inversio_crear:
    print(f"Number of exercises: {len(exercicis_inversio_crear)}")

    with open(basepath / f"inversio_crear_{syllable_type}.json", "w", encoding="utf-8") as f:
        json.dump(exercicis_inversio_crear, f, ensure_ascii=False, indent=2, sort_keys=True)